# Tutorial 2: Top-Hat Jet Models with Variable θ_c (Log-Normal)

> **Origin**: cleaned tutorial version of `Top_Hat_Models_LN.ipynb`  
> **Full production versions**: see `Top_Hat_Models_LN.ipynb` (multi-population loop) and `Top_Hat_Models.ipynb` (all three model variants: Epsilon, Flat, Log-Normal) in this folder.

This tutorial demonstrates how to run MCMC inference for GRB population models with **variable jet opening angles** (θ_c). Unlike Tutorial 1 (which uses a fixed θ_c = 3.4°), here we explore how to define this θ_c distribution from scratch when using a top-hat model in particular a **Log-Normal Distribution**.

## Key Differences from Tutorial 1

- **Parameters** There is not time evolution and we sample the luminosity directly. 
- **Geometric efficiency**: Must be computed from the θ_c distribution rather than a fixed value

## Tutorial Outline

1. Import libraries and set up environment
2. Define log-normal θ_c model
3. Run MCMC for a single model
4. Generate diagnostic plots
5. (at home) Run multiple population models
6. (at home) Compare results across different θ_c assumptions

Let's begin!

## Import Libraries

We import specialized modules handling the physics of top-hat jet models with variable θ_c distributions.

In [ ]:
# Auto-reload for interactive development
%load_ext autoreload
%autoreload 2

# Core imports
import  src.init
import  emcee
import  corner
import  numpy                   as np 
import  matplotlib.pyplot       as plt
from    math                    import inf
from    pathlib                 import Path

# Top-hat jet model specific modules
from    src.top_hat.montecarlo  import *
from    src.top_hat.plot        import TopHatPlotter

# Apply custom plotting style
plt.style.use('./configurations/style.mplstyle')

## Setup Directories and Global Parameters

In [ ]:
# Directory containing Fermi-GBM catalog data
datafiles   = Path("datafiles")

# Number of MCMC walkers
N_WALKERS   = 20

# Log-Normal θ_c Distribution Model

In this model, the jet opening angle θ_c follows a **log-normal distribution**:

$$
P(\theta_c | \theta_{c,\text{med}}, \sigma_{\theta_c}) = \frac{1}{\theta_c \sigma_{\theta_c} \sqrt{2\pi}} \exp\left(-\frac{(\ln \theta_c - \ln \theta_{c,\text{med}})^2}{2\sigma_{\theta_c}^2}\right)
$$

where:
- $\theta_{c,\text{med}}$ is the median opening angle (free parameter)
- $\sigma_{\theta_c}$ is the width parameter (typically fixed at 0.5 for this tutorial)

This model has **6 free parameters**:
1. `A_index`: Power-law index $k$ for energy distribution
2. `L_L0`: Log₁₀ isotropic energy scale
3. `L_mu_E`: Log₁₀ mean peak energy
4. `sigma_E`: Width of peak energy distribution
5. `theta_c_med`: **Median jet opening angle** (in degrees)
6. `fj`: **Jet fraction** - fraction of BNS mergers producing jets

## Define Prior, Likelihood, and Walker Initialization

The key difference from Tutorial 1 is the **geometric efficiency calculation**. Instead of using a fixed θ_c, we must integrate over the log-normal distribution to compute the average solid angle.

In [3]:
from src.top_hat.montecarlo import *

N_PARAMS        = 6
PARAM_NAMES     = ["A_index", "L_L0", "L_mu_E", "sigma_E", "theta_c_med", "fj"]

def log_prior_log_normal(thetas):
    # Change the variable name from theta_c_med_10 to theta_c_med
    A_index, L_L0, L_mu_E, sigma_E, theta_c_med, fj = thetas

    if not (1.5 < A_index       < 12): return -inf
    if not (-2  < L_L0          < 7): return -inf
    if not (0.1 < L_mu_E        < 7): return -inf
    if not (0   < sigma_E       < 2.5): return -inf
    if not (1   < theta_c_med   < 25): return -inf
    if not (0   < fj            < 1): return -inf
    
    return 0.0


from scipy.interpolate import interp1d
def create_geometric_efficiency_lognormal_interpolator(sigma_theta_c=0.5, n_points=200):
    """
    Create an interpolator for lognormal geometric efficiency.
    
    Call this once before MCMC, then use the returned function.
    """
    theta_c_med_grid = np.linspace(1, 25, n_points)
    
    efficiencies = np.array([
        _calculate_geometric_efficiency_lognormal_raw(t, sigma_theta_c) 
        for t in theta_c_med_grid
    ])
    
    return interp1d(theta_c_med_grid, efficiencies, kind='cubic', 
                    bounds_error=False, fill_value=(efficiencies[0], efficiencies[-1]))

def _calculate_geometric_efficiency_lognormal_raw(theta_c_med, sigma_theta_c=0.5):
    """Raw calculation - use interpolator version in MCMC."""
    mu      = np.log(theta_c_med)
    sigma   = sigma_theta_c * np.log(10)
    
    shape = sigma
    scale = np.exp(mu)

    theta_c_min = 1
    theta_c_max = 45

    cdf_max = lognorm.cdf(theta_c_max, s=shape, scale=scale)
    cdf_min = lognorm.cdf(theta_c_min, s=shape, scale=scale)
    norm    = cdf_max - cdf_min
    if norm <= 1e-9:
        return 0.0

    def integrand(theta_c_deg):
        # theta_c_deg in degrees, between 1 and 90
        theta_c_rad = np.deg2rad(theta_c_deg)
        detection_prob = 1.0 - np.cos(theta_c_rad)
        pdf = lognorm.pdf(theta_c_deg, s=shape, scale=scale) # use scipy's lognorm.pdf for numerical stability
        return detection_prob * pdf

    # integrate ordinary lognormal from small val to 90 deg
    geometric_eff_raw, _ = quad(integrand, theta_c_min, theta_c_max, epsabs=1e-8, epsrel=1e-8)
    # divide by CDF() to get truncated expectation
    return geometric_eff_raw / norm

geom_eff_interp = create_geometric_efficiency_lognormal_interpolator(sigma_theta_c=0.5)
def log_likelihood_log_normal(thetas, params_in, distances, k_interpolator, n_events=10_000):
    theta_c_med, fj             = thetas[4], thetas[5]
    
    gbm_eff                     = 0.6
    geometric_efficiency        = geom_eff_interp(theta_c_med)
    triggered_years             = params_in.triggered_years
    epsilon                     = geometric_efficiency * fj

    factor                      = 1 # Simulate less as the effieicny is already making this hella slow
    n_years                     = triggered_years / factor
    intrinsic_rate_per_year     = epsilon * len(params_in.z_arr) * gbm_eff
    expected_events             = intrinsic_rate_per_year * n_years
    #n_events                    = int(expected_events)
    
    results                     = simplified_montecarlo(thetas, n_events, params_in, distances, k_interpolator)
    
    trigger_mask, analysis_mask = apply_detection_cuts(results["p_flux"], results["E_p_obs"])
    
    if np.sum(analysis_mask) <= 3:
        return -inf, -inf, -inf, -inf, -inf
    
    triggered_events        = np.sum(trigger_mask)
    physics_efficiency      = triggered_events  / n_events
    
    predicted_detections    = expected_events * physics_efficiency
    observed_detections     = params_in.yearly_rate * params_in.triggered_years
    
    l1 = score_func_cvm(results["p_flux"][analysis_mask], params_in.pflux_data, params_in.rng)
    l2 = score_func_cvm(results["E_p_obs"][analysis_mask], params_in.epeak_data, params_in.rng)
    l3 = poiss_log(k=observed_detections, mu=predicted_detections)
        
    return l1 + l2 + l3, l1, l2, l3, physics_efficiency

# log_likelihood similar to flat case but uses calculate_geometric_efficiency_lognormal

def initialize_walkers_log_normal(n_walkers):
    np.random.seed(42)
    return np.column_stack([
        np.random.uniform(2, 3, n_walkers),
        np.random.uniform(2, 4.0, n_walkers),
        np.random.uniform(2.0, 4.0, n_walkers),
        np.random.uniform(0.2, 1, n_walkers),
        np.random.uniform(5, 10, n_walkers),  
        np.random.uniform(0.5, 1, n_walkers),
    ])

## Create Output Directory

We'll save all results (chains, plots, summaries) to a dedicated directory.

In [ ]:
# Create timestamped output directory for this run
output_dir_lm = src.init.create_run_dir(f"Paper_Results/Lognormal_Theta_FP/fiducial")

Loading existing directory  : Output_files/Paper_Results/Lognormal_Theta_FP/fiducial


## Run MCMC Sampling

Now we set up and run the MCMC:

1. **Initialize simulation** with Fermi-GBM catalog data
2. **Create interpolators** for fast likelihood evaluation
3. **Check for existing chains** to resume if interrupted
4. **Run MCMC** with parallel processing

**Note**: Start with fewer steps (e.g., 1,000) to test.

In [ ]:
# Where to find the datafiles and save the outputs
params = {
    "alpha"         : -0.67,    # 2/3 from synchrotron
    "beta_s"        : -2.59,    # Average value from GRBs
    "n"             : 2,        # Smoothly broken power law curvature
    "theta_c"       : 3.4,      # Ghirlanda half-angle of jet core (from GW170817)
    "theta_v_max"   : 10,       # Maximum viewing angle of the jet (in degrees)
    "z_model"      : 'fiducial_Hrad_A1.0'    #? Optional, model to use for redshift distribution
}
default_params, _, data_dict        = src.init.initialize_simulation(datafiles, params=params) #? Catalogue data from fermi GBM

print(f"Total events in the catalogue: {len(default_params.pflux_data)}")

n_walkers       = 20
n_steps         = 1_000

k_interpolator  = create_k_interpolator()

log_likelihood_default_in_log_normal = create_log_probability_function(
    log_prior_func          =   log_prior_log_normal,
    log_likelihood_func     =   log_likelihood_log_normal,
    params_in               =   default_params,
    k_interpolator          =   k_interpolator
)

initial_pos, n_steps, backend = check_and_resume_mcmc(
    filename                    =    output_dir_lm / "emcee.h5", 
    n_steps                     =   n_steps, 
    initialize_walkers_func     =   initialize_walkers_log_normal, 
    n_walkers                   =   n_walkers
) 

sampler = run_mcmc(
    log_probability_func    =   log_likelihood_default_in_log_normal,
    initial_walkers         =   initial_pos,
    n_iterations            =   n_steps,
    n_walkers               =   n_walkers,
    n_params                =   N_PARAMS,
    backend                 =   backend,
)

Using redshift model: samples_fiducial_Hrad_A1.0_BNSs_0.dat with 359079 BNSs.
Generating temporal interpolators... (this may take a moment)
Preparing catalogue with limits: {'F_LIM': 4, 'T90_LIM': 2, 'EP_LIM_UPPER': 10000, 'EP_LIM_LOWER': 50}
Triggered events: 310, Trigger years: 16.66, Yearly rate: 18.61 events/year
Total events in the catalogue: 268
Already completed this run


0it [00:00, ?it/s]


## Generate Diagnostic Plots

The `TopHatPlotter` class provides comprehensive visualization:

- **Corner plot**: Parameter distributions and correlations
- **Convergence plots**: Track MCMC evolution
- **Posterior predictive checks**: Compare model to data
- **Parameter summaries**: Median and credible intervals

All plots are automatically saved to the output directory.

In [ ]:
# plot
backend_ln = emcee.backends.HDFBackend(output_dir_lm / "emcee.h5")

plotter_log_normal = TopHatPlotter(
    backend     =   backend_ln,
    output_dir  =   output_dir_lm,
    model_type  =   "lognormal_theta_flat",
    burn_in     =   300,
    thin        =   15
)

# Generate all plots
distances = compute_luminosity_distance(default_params.z_arr)
summary = plotter_log_normal.generate_all_plots(
    mc_func=simplified_montecarlo,
    params_in=default_params,
    distances=distances,
    k_interpolator=k_interpolator,
    geometric_eff_func=geom_eff_interp
)

print(summary)  # Print LaTeX table

Generating corner plot...
(93320, 6)
Generating likelihood evolution...
Generating autocorrelation plot...
Generating corner plot for last two parameters...
Generating CDF comparison...
All plots saved to: Output_files/Paper_Results/Lognormal_Theta_FP/fiducial
\begin{tabular}{lc}
\hline
Parameter & Value \\
\hline
$k$ & $7.067_{-3.584}^{+3.417}$ \\
$\log_{10}(L_0)$ & $2.975_{-0.474}^{+0.542}$ \\
$\log_{10}(\mu_E)$ & $3.493_{-0.229}^{+0.667}$ \\
$\sigma_E$ & $0.429_{-0.110}^{+0.244}$ \\
$\theta_{c,\mathrm{med}}$ & $9.154_{-6.673}^{+9.872}$ \\
$f_j$ & $0.218_{-0.161}^{+0.442}$ \\
\hline
\end{tabular}
